In [ ]:
resss = []
objecttt = []
import numpy as np
import scipy
from scipy import stats
from copy import deepcopy
import math
import random as rn
import matplotlib.pyplot as plt
from scipy.optimize import least_squares


for epoch in range(100):
    global_max=0

    # Order Book for Hedging Class
    class LOB:   
        def __init__(self, midprice, volatility, perm_impact, spread, bp_distance):
            self.midprice = midprice
            self.volatility = volatility
            self.perm_impact = perm_impact
            self.spread = spread
            self.bp_distance = bp_distance

        def price_update(self, hedging_amount):
            dt_sec = 1
            random_bps = np.random.normal(0, self.volatility * np.sqrt(dt_sec/86400))
            psi = 0.1; eta = 1e-5; pmi = 0.005
            size = hedging_amount*dt_sec/86400
            cost = 1e-4*np.sign(hedging_amount) * (psi + eta * np.abs(hedging_amount))
            impact_bps = pmi*size
            dS = 1e-4 * (impact_bps + random_bps)
            self.midprice += dS
            return dS

        def market_impact(self, hedging_amount):      
            return hedging_amount/(10**6) * 0.5


    # Market-Maker Class
    class MarketMaker:
        def __init__(self, theta0, theta1, theta2, theta3, theta4, theta5, theta_sigma, inventory):
            self.theta0 = theta0
            self.theta1 = theta1
            self.theta2 = theta2
            self.theta3 = theta3
            self.theta4 = theta4
            self.theta5 = theta5
            self.theta_sigma = theta_sigma
            self.inventory = inventory
    #         self.hedging_amount = hedging_amount

        def policy_output(self):
            sign=np.sign(self.inventory[0])
            mu = (self.theta0 + self.theta1 * abs(self.inventory[0])**(1/5) + self.theta2 * abs(self.inventory[0])**(2/5) \
                  + self.theta3 * abs(self.inventory[0])**(3/5) + self.theta4 * abs(self.inventory[0])**(4/5) \
                  + self.theta5 * abs(self.inventory[0]))*sign
            sigma = self.theta_sigma
            hedging_amount = np.random.normal(loc = mu, scale = sigma, size = 1)[0] 
            return hedging_amount

        def reward_output(self, market_impact, price_change, spread, volatility, mm_profit, hedging_amount): 
            reward = mm_profit + (self.inventory + hedging_amount) * price_change \
            - abs(self.inventory + hedging_amount) * volatility - abs(hedging_amount) * (spread * 0.5 + abs(market_impact))      
            return reward



    class Client_Trader():
        def __init__(self):
            self.nothing = 0


        def getAmount(self):
            amountsList = [1000000, 3000000, 5000000]
            p = [0.8,  0.15, 0.05]
            idx = np.random.choice(len(p), p=p)
            amount = np.array(rn.choice([-1,1]) * amountsList[idx])
            return np.array([float(amount)])
            

    def gradient_log(theta0, theta1, theta2, theta3, theta4, theta5, theta_sigma, inventory, hedging_amount): # sigma = theta2

        sign=np.sign(inventory)

        gr0 = sign * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                   /(theta_sigma**2)
        gr1 = sign * abs(inventory)**(1/5) * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                                   /(theta_sigma**2)
        gr2 = sign * abs(inventory)**(2/5) * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                                   /(theta_sigma**2)
        gr3 = sign * abs(inventory)**(3/5) * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                                   /(theta_sigma**2)
        gr4 = sign * abs(inventory)**(4/5) * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                                   /(theta_sigma**2)
        gr5 = sign * abs(inventory) * (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))) \
                            /(theta_sigma**2)
        gr_sigma = -1/theta_sigma + (hedging_amount - sign*(theta0 + theta1 * abs(inventory)**(1/5) + theta2 * abs(inventory)**(2/5) \
                          + theta3 * abs(inventory)**(3/5) + theta4 * abs(inventory)**(4/5) + theta5 * abs(inventory))**2) \
                                      /(theta_sigma**3)
        return [gr0, gr1, gr2, gr3, gr4, gr5, gr_sigma]   

    def v_approximarion(weights, px0, px1, px2, px3, px4):
        result = weights[0] * px0 + weights[1] * px1 + weights[2] * px2 + weights[3] * px3 + weights[4] * px4

        return result

    def func(params, xdata, ydata):
        return (ydata - np.dot(xdata, params))

    def values_matrix(states):
        p0 = []
        p1 = []
        p2 = []
        p3 = []
        p4 = []
        for i in range(len(states)):
            sign = np.sign(states[i][0])
            p0.append(1 * 1/abs(states[i][0]+1) * sign)
            p1.append(abs(states[i][0])**(1/16) * 1/abs(states[i][0]) * sign) # +1 к абсу
            p2.append(abs(states[i][0])**(1/8) * 1/abs(states[i][0]) * sign)
            p3.append(abs(states[i][0])**(1/4) * 1/abs(states[i][0]) * sign)
            p4.append(abs(states[i][0])**(1/2) * 1/abs(states[i][0]) * sign)

        observed_data = [p0, p1, p2, p3, p4]

        return observed_data


    midprice = 1
    perm_impact = 0.005
    bp_distance = 1/10000
    volatility = 50 * bp_distance
    spread = 5 * bp_distance
    theta0 = 1000
    theta1 = -0.00001
    theta2 = -0.00001
    theta3 = -0.00001
    theta4 = -0.00001
    theta5 = -0.0000015
    theta_sigma = 250
    weights = np.array([0.00001] * 5)
    inventory = 0
    hedging_amount = 0 


    # Initial parameters for training (subject to change):
    tol = 10**(-4)
    grad_J = [+np.inf, +np.inf, +np.inf, +np.inf, +np.inf, +np.inf, +np.inf] 
    norm_grad_J = (grad_J[0]**2 + grad_J[1]**2 + grad_J[2]**2 + grad_J[3]**2 + grad_J[4]**2 + \
                   grad_J[5]**2 + grad_J[6]**2)**0.5 #+ J[2]**2 + J[3]**2
    time = 700
    alpha = 2**(-8)
    beta = 2**(-10)
    gamma = 0.995
    theta1_benchmark = -10**(-6)
    theta2_benchmark = 10**(-5)
    steps = 0
    objective_values = []
    residuals = []

    Order_book = LOB(midprice, volatility, perm_impact, spread, bp_distance)
    Client = Client_Trader()
    Market_Maker = MarketMaker(theta0, theta1, theta2, theta3, theta4, theta5, theta_sigma, inventory)


    while steps<1600:#norm_grad_J > tol:
        if steps != 0 and steps%200 == 0:
            alpha = alpha/2
        check = []
        states = [] 
        actions = [] 
        rewards = [] 
        Order_book.midprice = 1 
        Market_Maker.inventory = 0 
        mm_profit = 0 
        avg_price = 0 
        # Trajectory
        for i in range(time):    
            # LOB updates based on our previous actions
            price_delta = Order_book.price_update(hedging_amount) 

            # Client trades x
            x = Client.getAmount() 
            

            check.append(x)
            if Market_Maker.inventory > 0: 
                if x > 0: 
                    priceWithSpread = Order_book.midprice + (spread/2)
                    if x <= Market_Maker.inventory:
                        mm_profit = x * (priceWithSpread - avg_price)
                        Market_Maker.inventory -= x 
                    else:
                        mm_profit = Market_Maker.inventory * (priceWithSpread - avg_price)
                        Market_Maker.inventory -= x 
                        avg_price = deepcopy(priceWithSpread)
                else: 
                    mm_profit = 0
                    priceWithSpread = Order_book.midprice - (spread/2)
                    Market_Maker.inventory -= x 
                    avg_price = (avg_price * (Market_Maker.inventory-abs(x)) + deepcopy(priceWithSpread) * abs(x))/Market_Maker.inventory

            elif Market_Maker.inventory < 0: 
                if x < 0: 
                    priceWithSpread = Order_book.midprice - (spread/2)
                    if abs(x) <= abs(Market_Maker.inventory): 
                        mm_profit = x * (avg_price - priceWithSpread)
                        Market_Maker.inventory -= x 
                    else:
                        mm_profit = Market_Maker.inventory * (avg_price - priceWithSpread)
                        Market_Maker.inventory -= x 
                        avg_price = deepcopy(priceWithSpread)
                else: 
                    mm_profit = 0
                    priceWithSpread = Order_book.midprice + (spread/2)
                    Market_Maker.inventory -= x 
                    avg_price = (avg_price * abs(Market_Maker.inventory+x) + x * priceWithSpread)/abs(Market_Maker.inventory)    
            else: 
                if x < 0: 
                    avg_price = Order_book.midprice - (spread/2)
                if x > 0:
                    avg_price = Order_book.midprice + (spread/2)
                else:
                    avg_price = 0    
                Market_Maker.inventory -= x
                mm_profit = 0


            inventory = deepcopy(Market_Maker.inventory)
            hedging_amount = Market_Maker.policy_output() 
            mi = Order_book.market_impact(hedging_amount) 
            R = Market_Maker.reward_output(mi, price_delta, spread, volatility, mm_profit, hedging_amount)/(10**10)

            states.append(inventory)
            actions.append(hedging_amount)
            rewards.append(R)

            Market_Maker.inventory += hedging_amount


        if abs(sum(check)) > 100e6:
            continue

        
        G = []
        for i in range(len(rewards)): 
            g_k = 0
            for j in range(i, len(rewards)): 
                g_k += rewards[j] * gamma**(j - i)
            G.append(g_k[0])
        




        objective_values.append(sum(G))

        # Least squares optimisation of weights
        matrix = values_matrix(states)
        observed_data = np.transpose(np.array(matrix))
        result = least_squares(func, weights, args=(observed_data, np.array(G)), method='trf', loss = 'cauchy')
        weights = result.x
        residuals.append(2 * result.cost)


        for i in range(len(G)):
            if i < len(G)-1:
                delta = G[i] + gamma * v_approximarion(weights, matrix[0][i+1], matrix[1][i+1], \
                                                       matrix[2][i+1], matrix[3][i+1], matrix[4][i+1]) - \
                        v_approximarion(weights, matrix[0][i], matrix[1][i], matrix[2][i], matrix[3][i], matrix[4][i])
            else:
                delta = G[i] - v_approximarion(weights, matrix[0][i], matrix[1][i], matrix[2][i], matrix[3][i], matrix[4][i])
      
            temp = gradient_log(Market_Maker.theta0, Market_Maker.theta1, Market_Maker.theta2, Market_Maker.theta3 \
                                , Market_Maker.theta4, Market_Maker.theta5, Market_Maker.theta_sigma, states[i], actions[i])
            
            norm_temp=(temp[0]**2+temp[1]**2+temp[2]**2+temp[3]**2+temp[4]**2+temp[5]**2+temp[6]**2)**0.5
            Market_Maker.theta0 = Market_Maker.theta0 + alpha * (gamma**i) * delta * temp[0]/norm_temp
            Market_Maker.theta1 = min(0, Market_Maker.theta1 + alpha * (gamma**i) * delta * temp[1]/norm_temp)
            Market_Maker.theta2 = min(0, Market_Maker.theta2 + alpha * (gamma**i) * delta * temp[2]/norm_temp)
            Market_Maker.theta3 = min(0, Market_Maker.theta3 + alpha * (gamma**i) * delta * temp[3]/norm_temp)
            Market_Maker.theta4 = min(0, Market_Maker.theta4 + alpha * (gamma**i) * delta * temp[4]/norm_temp)
            Market_Maker.theta5 = min(0, Market_Maker.theta5 + alpha * (gamma**i) * delta * temp[5]/norm_temp)
            Market_Maker.theta_sigma = max(1, Market_Maker.theta_sigma + alpha * (gamma**i) * delta * temp[6]/norm_temp)
            
        steps += 1    

        if steps % 100 == 0:
            print('Epoch: ', epoch+1)
            print(actions[-1]/states[-1])
            print('Objective function: ', objective_values[-1])
            print('Theta0: ', Market_Maker.theta0)
            print('Theta1: ', Market_Maker.theta1)
            print('Theta2: ', Market_Maker.theta2)
            print('Theta3: ', Market_Maker.theta3)
            print('Theta4: ', Market_Maker.theta4)
            print('Theta5: ', Market_Maker.theta5)
            print('Theta_sigma: ', Market_Maker.theta_sigma)
            print('Steps taken: ', steps)
            plt.plot(range(steps), objective_values)
            plt.show()
            print('==============================================')
    resss.append(residuals)# EXPORT AFTER SIMULATION
    objecttt.append(objective_values)# EXPORT AFTER SIMULATION
